# Format signal component results

Pandas code to produce the tables in the appendix of `MCC`.

After formatting, tables are fed into [Tables Generator](https://www.tablesgenerator.com/latex_tables) to produce final latex version.

In [1]:
import pandas as pd
from graspologic.datasets import load_mice

## Table 1: Signal Edges

The top 20 signal edges (out of 54,946 total edges) ranked by the order of their Holm--Bonferroni corrected $p$-value. Eleven of the top 20 signal edges are adjacent to either the left or right hemisphere corpus callosum.

In [2]:
mice = load_mice()
mice.atlas.head()

,ROI,Structure,Abbreviation,c_R,c_G,c_B,c_A,Level_1,Level_2,Level_3,Level_4,Macrostructure
0,1,Cingulate_Cortex_Area_24a,A24a,255,0,0,255,forebrain,secondary_prosencephalon,isocortex,cingulate_cortex,isocortex
1,2,Cingulate_Cortex_Area_24a_prime,A24aPrime,255,124,0,255,forebrain,secondary_prosencephalon,isocortex,cingulate_cortex,isocortex
2,3,Cingulate_Cortex_Area_24b,A24b,255,248,0,255,forebrain,secondary_prosencephalon,isocortex,cingulate_cortex,isocortex
3,4,Cingulate_Cortex_Area_24b_prime,A24bPrime,138,255,0,255,forebrain,secondary_prosencephalon,isocortex,cingulate_cortex,isocortex
4,5,Cingulate_Cortex_Area_29a,A29a,14,255,0,255,forebrain,secondary_prosencephalon,isocortex,cingulate_cortex,isocortex


In [3]:
def lookup_roi_name(roi):
    roi -= 1
    hemisphere = "R" if roi // 166 else "L"
    roi = roi % 166
    structure = mice.atlas["Structure"].values[roi]
    structure = " ".join(structure.split("_"))
    return f"{structure} ({hemisphere})"

In [4]:
signal_edges = pd.read_csv("../results/signal_edges.csv")

# Get the top 20 strongest signal edges
strong_signal_edges = signal_edges.head(20)
strong_signal_edges["ROI_1"] = strong_signal_edges["ROI_1"].apply(lookup_roi_name)
strong_signal_edges["ROI_2"] = strong_signal_edges["ROI_2"].apply(lookup_roi_name)

strong_signal_edges.drop(["pvalue", "significant"], axis=1, inplace=True)
strong_signal_edges.columns = ["Vertex 1", "Vertex 2", "statistic", "p-value"]

strong_signal_edges["statistic"] = strong_signal_edges["statistic"].apply(lambda x: f"{x:.3f}")
strong_signal_edges["p-value"] = strong_signal_edges["p-value"].apply(lambda x: f"{x:.3f}")

strong_signal_edges

,Vertex 1,Vertex 2,statistic,p-value
0,Corpus Callosum (L),Striatum (R),0.717,0.054
1,Corpus Callosum (L),Internal Capsule (R),0.699,0.073
2,Corpus Callosum (L),Reticular Nucleus of Thalamus (R),0.698,0.074
3,Corpus Callosum (L),Zona Incerta (R),0.686,0.092
4,Septum (R),Corpus Callosum (R),0.671,0.118
5,Lateral Ventricle (L),Striatum (R),0.667,0.125
6,Striatum (L),Striatum (R),0.664,0.132
7,Corpus Callosum (L),Ventral Thalamic Nuclei (R),0.663,0.133
8,Hippocampus (L),Middle Cerebellar Peduncle (L),0.658,0.144
9,Caudomedial Entorhinal Cortex (R),Ventral Hippocampal Commissure (R),0.656,0.150


In [5]:
strong_signal_edges.round(3).to_csv("../results/formatted_tables/edges.csv", index=False)

## Table 2: Signal Vertices

The top 20 signal vertices (out of 332 total vertices) ranked by the order of their Holm--Bonferroni corrected $p$-values. Pillai's trace and approximate $F$ statistic (along with degrees of freedom) as calculated by one-way MANOVA are also reported. The corpus callosum in the left and right hemisphere are the top two signal vertices.

In [6]:
signal_vertices = pd.read_csv("../results/signal_vertices.csv")

# Get the top 10 strongest signal edges
strong_signal_vertices = signal_vertices.head(20)
strong_signal_vertices["ROI"] = strong_signal_vertices["ROI"].apply(lookup_roi_name)
strong_signal_vertices

strong_signal_vertices.drop(["num.df", "den.df", "pvalue", "significant"], axis=1, inplace=True)
strong_signal_vertices.columns = ["Vertex", "Pillai", "F(15, 78)", "p-value"]

# pd.set_option('display.float_format', '{:.3g}'.format)
strong_signal_vertices["Pillai"] = strong_signal_vertices["Pillai"].apply(lambda x: f"{x:.3f}")
strong_signal_vertices["F(15, 78)"] = strong_signal_vertices["F(15, 78)"].apply(lambda x: f"{x:.3f}")
strong_signal_vertices["p-value"] = strong_signal_vertices["p-value"].apply(lambda x: f"{x:.3g}")
strong_signal_vertices

,Vertex,Pillai,"F(15, 78)",p-value
0,Corpus Callosum (L),2.591,32.914,5.09e-25
1,Corpus Callosum (R),2.556,29.951,1.09e-23
2,Fimbria (L),2.440,22.637,7.27e-20
3,Secondary Motor Cortex (L),2.438,22.544,8.21e-20
4,Midbrain Reticular Nucleus (R),2.430,22.161,1.38e-19
5,Substantia Nigra (R),2.305,17.254,2.2e-16
6,Internal Capsule (R),2.304,17.229,2.29e-16
7,Secondary Motor Cortex (R),2.297,16.989,3.4e-16
8,Cerebral Peduncle (R),2.247,15.511,4.34e-15
9,Internal Capsule (L),2.238,15.266,6.71e-15


In [7]:
strong_signal_vertices.to_csv("../results/formatted_tables/vertices.csv", index=False)

## Table 3: Bilateral Signal Pairs

The top 10 bilateral signal vertex pairs (out of 166 total vertex pairs) ranked by the order of their average total rank.

In [8]:
def roi_name_no_hemisphere(roi):
    name = lookup_roi_name(roi)
    name = " ".join(name.split(" ")[:-1])
    return name

In [9]:
def bilateral_twin(roi):
    left = True
    if roi > 166:
        left = False
        return roi - 166, left
    else:
        return roi + 166, left

In [10]:
def bilateral_ranks(roi):

    bilateral_roi, left = bilateral_twin(roi)
    if left:
        left_roi = roi
        right_roi = bilateral_roi
    else:
        left_roi = bilateral_roi
        right_roi = roi
        
    left_rank = bilateral.query(f"ROI == {left_roi}")["rank"].values[0]
    right_rank = bilateral.query(f"ROI == {right_roi}")["rank"].values[0]

    left_pval = bilateral.query(f"ROI == {left_roi}")["holm_pvalue"].values[0]
    right_pval = bilateral.query(f"ROI == {right_roi}")["holm_pvalue"].values[0]


    return left_pval, left_rank, right_pval, right_rank

In [11]:
def avg_rank(row):
    return (row.left_rank + row.right_rank) / 2

In [12]:
bilateral = signal_vertices.copy()
bilateral = bilateral.drop(["pillai", "F", "num.df", "den.df", "pvalue", "significant"], axis=1)
bilateral["rank"] = bilateral["holm_pvalue"].rank().astype(int)
bilateral["name"] = bilateral["ROI"].apply(roi_name_no_hemisphere)

bilateral["left_pval"], bilateral["left_rank"], bilateral["right_pval"], bilateral["right_rank"] = zip(*bilateral["ROI"].apply(bilateral_ranks))
bilateral = bilateral.drop(["ROI", "holm_pvalue", "rank"], axis=1)
bilateral = bilateral.drop_duplicates()

bilateral["left_pval"] = bilateral["left_pval"].apply(lambda x: f"{x:.3g}")
bilateral["right_pval"] = bilateral["right_pval"].apply(lambda x: f"{x:.3g}")

bilateral["avg_rank"] = bilateral.apply(avg_rank, axis=1)
bilateral = bilateral.sort_values(["avg_rank"])

# bilateral.head(10)
bilateral.head(10).to_csv("../results/formatted_tables/vertices_bilateral.csv", index=False)

## Table 4: Signal Communities

The top 10 signal communities (out of 105 total communities) ranked by the order of their Holm--Bonferroni corrected $p$-values as calculated by the Multivariate Weighted method.

In [13]:
signal_communities = pd.read_csv("../results/signal_communities.csv")

signal_communities = signal_communities.drop(["pvalue", "significant"], axis=1)

signal_communities["statistic"] = signal_communities["statistic"].apply(lambda x: f"{x:.3f}")
signal_communities["holm_pvalue"] = signal_communities["holm_pvalue"].apply(lambda x: f"{x:.3g}")

signal_communities.head()

,Community 1,Community 2,statistic,holm_pvalue
0,White Matter (R),White Matter (R),0.888,6.14e-06
1,Hindbrain (R),White Matter (R),0.872,7.87e-06
2,White Matter (L),White Matter (L),0.871,8.02e-06
3,Subpallium (R),White Matter (R),0.852,1.07e-05
4,Hindbrain (L),White Matter (L),0.848,1.14e-05


In [14]:
signal_communities.head(10).to_csv("../results/formatted_tables/communities.csv", index=False)